# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/workspace/event_data/2018-11-27-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    {'class' : 'SimpleStrategy', 'replication_factor' : 1}"""
    )
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
"""
Songplays table that will dispaly song info based on a specific session and position in a playlist
Columns
    - song artist, song title and song length are the data points to be returned in the query
    - sessionId and itemInSession are what we're filtering the data on. sessionId represents a unique user session so that will be our partition key.
      Since we want to view the fourth song in a playlist, itemInSession is the cluserting column.
"""

query1 = "CREATE TABLE IF NOT EXISTS songplays "
query1 += "( sessionId int, itemInSession int, artist text, length float, song text, PRIMARY KEY (sessionId, itemInSession))"

In [9]:
"""
Song_playlist_session table that will display song information that certain users have listened to
Columns
    - artist name, song name, user first and last name are the data points to be returned in the query
    - userId and sessionId represent the composite partition key as they should group the data more evenly than just one or the other
    - itemInSession is the clustering column because we wish to sort the results by it. Additionally, information will be lost if
      this isn't included because each data point is unique on userId, sessionId and itemInSession

"""

query2 = "CREATE TABLE IF NOT EXISTS song_playlist_session "
query2 += "(artist text, firstName text, itemInSession int, lastName text, sessionId int, song text, userId int, PRIMARY KEY ((userId, sessionId), itemInSession))"


In [10]:
"""
User_listens table that will dispaly user information and songs they've listened to
Columns
    - firstName and lastName are the data points returned in the query
    - Song is the partition key because that is the first and only column we filter on in the query
    - userId is the clustering column because the data is unique compositely on song and userId
"""

query3 = "CREATE TABLE IF NOT EXISTS user_listens "
query3 += "(firstName text, lastName text, userId int, song text, PRIMARY KEY (song, userId))"

In [11]:
table_names = ['songplays', 'song_playlist_session', 'user_listens']
for table in table_names:
    try:
        session.execute(f"DROP TABLE IF EXISTS {table}")
    except Exception as e:
        print(e)
try:
    session.execute(query1)
    session.execute(query2)
    session.execute(query3)
except Exception as e:
    print(e)

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query1 = "INSERT INTO songplays (artist, itemInSession, length, sessionId, song) VALUES (%s, %s, %s, %s, %s)"
        query2 = "INSERT INTO song_playlist_session (artist, firstName, itemInSession, lastName, sessionId, song, userId) VALUES (%s, %s, %s, %s, %s, %s, %s)"
        query3 = "INSERT INTO user_listens (firstName, lastName, userId, song) VALUES (%s, %s, %s, %s)"
        itemInSession = int(line[3])
        length = float(line[5])
        sessionId = int(line[8])
        userId = int(line[10])
        try:
            session.execute(query1, (line[0], itemInSession, length, sessionId, line[9]))
            session.execute(query2, (line[0], line[1], itemInSession, line[4], sessionId, line[9], userId))
            session.execute(query3, (line[1], line[4], userId, line[9]))
        except Exception as e:
            print(e)

#### Do a SELECT to verify that the data have been inserted into each table

In [13]:
for table in table_names:
    results = session.execute(f"SELECT * FROM {table} LIMIT 5")
    for row in results:
        print(row)

Row(sessionid=23, iteminsession=0, artist='Regina Spektor', length=191.08526611328125, song='The Calculation (Album Version)')
Row(sessionid=23, iteminsession=1, artist='Octopus Project', length=250.95791625976562, song='All Of The Champs That Ever Lived')
Row(sessionid=23, iteminsession=2, artist='Tegan And Sara', length=180.06158447265625, song='So Jealous')
Row(sessionid=23, iteminsession=3, artist='Dragonette', length=153.39056396484375, song='Okay Dolores')
Row(sessionid=23, iteminsession=4, artist='Lil Wayne / Eminem', length=229.58975219726562, song='Drop The World')
Row(userid=58, sessionid=768, iteminsession=0, artist='System of a Down', firstname='Emily', lastname='Benson', song='Sad Statue')
Row(userid=58, sessionid=768, iteminsession=1, artist='Ghostland Observatory', firstname='Emily', lastname='Benson', song='Stranger Lover')
Row(userid=58, sessionid=768, iteminsession=2, artist='Evergreen Terrace', firstname='Emily', lastname='Benson', song='Zero')
Row(userid=85, session

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [14]:
query1 = "SELECT artist, song, length FROM songplays WHERE sessionId = 338 and itemInSession=4"
try:
    result1 = session.execute(query1)
    for row in result1:
        print(row.artist, row.song, row.length)
except Exception as e:
    print(e)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [17]:
query2 = "SELECT artist, song, firstname, lastname FROM song_playlist_session WHERE userId=10 and sessionId=182"
try:
    result2 = session.execute(query2)
    for row in result2:
        print(row.artist, row.song, row.firstname, row.lastname)
except Exception as e:
    print(e)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [20]:
query3 = "SELECT firstName, lastName FROM user_listens WHERE song = 'All Hands Against His Own'"
try:
    result3 = session.execute(query3)
    for row in result3:
        print(row.firstname, row.lastname)
except Exception as e:
    print(e)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [21]:
for table in table_names:
    try:
        session.execute(f"DROP TABLE IF EXISTS {table}")
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()